In [2]:
texts_raw = '''
this is good
real bad
I like what happned
i hated that person
cats have 4 legs
I have shorted the stock, cuz i see a slowdown in business
'''

texts = texts_raw.split('\n')
texts = [i for i in texts if i != '']
print(texts)

['this is good', 'real bad', 'I like what happned', 'i hated that person', 'cats have 4 legs', 'I have shorted the stock, cuz i see a slowdown in business']


# Loading 300k tweets and getting sentiment on each one then saving that to CSV.


In [13]:
import pandas as pd

# Load 300K Tweets zm-6-2019-04-01-2022-12-11-all-zm-tweets.jsonl
file_path = 'zm-6-2019-04-01-2022-12-11-all-zm-tweets.jsonl' #full file. 
# file_path = 'zm-tweets-jan to april 2020.csv' # testing smaller file
# file_path = 'zm_mismatch_df.csv'  # testing on mismatch only 5k
df = pd.read_csv(file_path)
# df = pd.read_json(file_path)




In [14]:
df.shape

(1352, 38)

In [22]:
# temp --- getting 5k tweets 
texts = df['clean_tweet']

texts = list(texts)

In [ ]:

# clean tweets and create a new col clean_tweets

# filter tweets that have cashtags that are 3 or lower (will remove alot of the spam and news report tweets)

# count remaining tweets.

In [27]:
# WORKING WITH CUDA and adding 300k tweets to test

# code wtih cuda reporting.

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:" if cuda_available else "CUDA not available")

# Load the RoBERTa tokenizer and model
model_name = f'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to("cuda" if cuda_available else "cpu")

# Create a list of 17,000 sample texts (replace this with your actual data)
# texts = ["Sample text {}".format(i) for i in range(17000)] #commented out to use my own text

# Set the batch size (you may need to adjust this based on your GPU memory)
batch_size = 32

# Function to classify sentiment in a batch of texts
def classify_sentiment(texts_batch):
    print(texts_batch)
    encoded_input = tokenizer(texts_batch, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {key: value.to("cuda" if cuda_available else "cpu") for key, value in encoded_input.items()}
    with torch.no_grad():
        logits = model(**encoded_input).logits
    probabilities = torch.softmax(logits, dim=1)
    print(probabilities)
    # labels = ["NEGATIVE" if p < 0.5 else "POSITIVE" for p in probabilities[:, 1]]
    max_indices = torch.argmax(probabilities, dim=1)
    # Map indices to labels
    labels_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    labels = [labels_map[index.item()] for index in max_indices]
    return labels
    # return probabilities

# Process the texts in batches
sentiments = []
num_batches = len(texts) // batch_size + int(len(texts) % batch_size > 0)

for i in range(0, len(texts), batch_size):
    texts_batch = texts[i:i + batch_size]
    sentiments_batch = classify_sentiment(texts_batch)
    sentiments.extend(sentiments_batch)
    
    # Print progress
    batches_completed = i // batch_size + 1
    batches_left = num_batches - batches_completed
    print(f"Batch {batches_completed}/{num_batches} completed. Batches left: {batches_left}")

# Print the first 10 sentiment labels
print(sentiments[:50])



CUDA available:


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['Four reasons a company falls into my too hard pile\n\n- industry (ex. energy)\n- Can\'t understand the business (ex. $DDOG)\n- region I can\'t trust or am ill-informed (ex. China)\n- followed too closely (ex. $ZM)\n\nWhy do you put stocks in the "too hard" pile?\n\nhttps://t.co/j61LV64yXP', "Zoom $ZM settled +4.4% to $141.89, helped via a Cantor Fitzgerald 'overweight to $150'. The privacy/security issue remains unresolved, and I see the stock as s/t toxic. https://t.co/IjcpcgNHfu", 'Yikes! $ZM https://t.co/UNWH5LexEd', 'I don’t understand the allure of $ZM in teaching.  Why not just make a video module instead? Youtube has taught millions of people this way for years.. why mess with the formula? The', '@stocktrader300 You can do a $ZM call &amp; be more effective, efficient, and a profit center vs. cost to company: $UBER ride, plane ticket, hotel room ,meals,etc..  business will change &amp; $ZM remains a powerful new paradigm.', 'Mr. Market pays roughly $1,000 per subscriber for $N

In [25]:
df['sentiment_r_latest'] = sentiments

In [26]:
df.to_csv('mismatch_w_r_latest_sentiment.csv')

In [ ]:
# Add sentiment back into main df


# save main 300ktweets_with_sentiment to csv.

In [15]:
import torch
print(torch.version.cuda)

11.3
